# Descarga de PDBs de la proteína CDK2
Esta libreta de Jupiter permite descargar las estructuras cristalográficas disponibles de una proteína dada una secuencia de UniProt, un valor de identidad y un valor de covertura.

In [61]:
from prody import *
from pylab import *
import pickle

/home/joel/anaconda3/envs/mds/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Secuencia de la proteína
Se obtiene define la secuencia de la proteína: P24941.

In [72]:
# Secuencia de la CDK2 de UniProt
import requests
from Bio import SeqIO

# Secuencia P24941 (CDK2_HUMAN)
uniprot_id = "P24941"
# Descargamos el fasta de UniProt
url_fasta = requests.get("https://www.uniprot.org/uniprot/" + uniprot_id + ".fasta")
file_name_fasta = './B_DATOS/' + uniprot_id + '.fasta'
open(file_name_fasta, 'wb').write(u.content)
# Leemos la secuenciade aminoácidos
handle = SeqIO.read(open(file_name_fasta),'fasta')
cdk2_P24941 = str(handle.seq)

cdk2_P24941 = "MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNHPNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHSHRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYYSTAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSFPKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL"

### BLASTp de la secuencia en el PDB
Descomentar el código para volver a ejecutar, o en su defecto, recuperar el archivo de blast guardado en el directorio.

In [73]:
## Hacemos un blast
blast_record = blastPDB(cdk2_P24941)

## Lo guardamos para poder usarlo y no tener que rehacerlo
pickle.dump(blast_record, open('B_DATOS/cdk2_blast_record.pkl', 'wb'))

# Para abrirlo:
blast_record = pickle.load(open('B_DATOS/cdk2_blast_record.pkl', 'rb'))

@> Blast searching NCBI PDB database for "MENFQ..."
@> Blast search completed in 21.0s.                 


Se obtienen los "hits" del BLAST según un valor de identidad especificado. También se obtiene una lista con los identificadores de dichos hits.

In [74]:
pdbids = blast_record.getHits(percent_identity=95) # Devuelve un diccionario con cada proteína hit
len(pdbids) # 391 proteínas tienen un 95% de identidad con la seucuencia
pdbids_list = list(pdbids.keys()) # lista de pdb_ids de las estructuras de CDK2
len(pdbids_list)

408

### Lista de identificadores de los PDB hits
Se guarda un archivo csv con todos los identificadores. Descomentar el código para volver a guardar.  
**Se identificaron 391 PDB_ids.**

In [67]:
# Guarda la lista de PDB IDs de los hits
import csv

num_pdb_ids = len(pdbids_list); num_pdb_ids
#with open('CDK2_pdb_' + str(num_pdb_ids) + '_IDs_P24941.csv', 'w') as myfile:
 # wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
 # wr.writerow(pdbids_list)

408

## Descarga de los archivos PDB
El siguiente ciclo descarga cada uno de los archivos PDB de la lista de identificadores PDB.
* Descarga el archivo PDB completo
* Descarga únicamente la cadena protéica del PDB con mayor identidad a la secuencia de UniProt dada
* Descarga cualquier ligando asociado a la cadena protéica de mayor identidad.

In [6]:
# Directorios
dir_pdbs_completos = './PDB_CDK2_completos/'
dir_pdbs_cadena = './PDB_CDK2_chains/'
dir_pdb_ligandos = './PDB_CDK2_ligands_chainA/'

best_id = blast_record.getBest()['pdb_id']
chain_best_id = pdbids[best_id]['chain_id']
ref_struct = parsePDB(best_id, folder = dir_pdbs_completos).select('protein and chain ' + chain_best_id)

# Esta lista es creada a aprtir de los hits del blast
### pdbids_list = ["1pye", "1jvp", "5ang"] # tienen un overlap menor al 90%
### pdbids_list = ["5ang"]

for pdb_id in pdbids_list:
    try:
        # PROTEÍNA cadena única
        chain_id = pdbids[pdb_id]['chain_id'] # Obtiene el id de la cadena correspondiente a la mejor estructura empatada según el BLAST
        pdb_cry = parsePDB(pdb_id, folder = dir_pdbs_completos) # Obtiene del pdb la estructura y la guarda en el folder
        pdb_chain = pdb_cry.select('protein and chain ' + chain_id) # selecciona de la proteína a la cadena que el BLAST indicó
        pdb_alg = matchAlign(pdb_chain, ref_struct, overlap=85) #Retorna un tuple con la estructura alineada en el primer índice
        ################ OJO: debo considerar si es buena o mala idea dejar estructuras con menos del 90 o 95% de OVERLAP #####
        protein = pdb_alg[0]
    except Exception as e:
        print(e, "Error al alinear:", pdb_id)
        continue
    else:
        repr(protein)
        if protein: # si hubo una cadena, la guarda
            protein.setChids("A") # fuerza que la cadena sea A para homogenizar si es necesario
            writePDB(dir_pdbs_cadena + pdb_id + '_A.pdb', protein)
        # LIGANDOS (Si los hay)
        ligand = pdb_cry.select('not protein and not water and chain ' + chain_id)
        repr(ligand)
        if ligand: # si hubo una cadena, la guarda
            #ligand.setChids("A")
            writePDB(dir_pdb_ligandos + pdb_id + '_lig.pdb', ligand)
            
# Se obtuvieron 390 estructuras dejando fuera a 5ANG que tuvo un overlap menor al 50%

@> PDB file is found in working directory (PDB_CDK2_completos/1fin.pdb.gz).
@> 9476 atoms and 1 coordinate set(s) were parsed in 0.12s.
@> PDB file is found in working directory (PDB_CDK2_completos/5ang.pdb.gz).
@> 2256 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> Checking AtomGroup 5ang: 3 chains are identified
@> Checking AtomGroup 1fin: 1 chains are identified
@> Trying to match chains based on residue numbers and names:
@>   Comparing Chain A from 5ang (len=127) and Chain A from 1fin (len=298):
@> 	Failed to match chains (seqid=100%, overlap=43%).
@>   Comparing Chain A from 5ang (len=5) and Chain A from 1fin (len=298):
@> 	Failed to match chains (seqid=100%, overlap=2%).
@>   Comparing Chain A from 5ang (len=136) and Chain A from 1fin (len=298):
@> 	Match: 136 residues match with 100% sequence identity and 46% overlap.
@> Alignment is based on 136 atoms matching 'calpha'.
@> Before alignment RMSD: 268.76
@> After alignment  RMSD: 1.39


In [6]:
# blast_record.writeSequences("a.d", blast_record.getHits())